# Topic Modelling on Program Source Code
---
By Kishalay Banerjee, Dan Jones and Sam Harding

## Generating the Dataset

TODO: Import dataset.py and explain how it's used + what it does.

TODO: Add Sam's scrub function in here?

## Topic Modelling on Individual Source Files

TODO:
  - Copy work from documentation/daniel-jones.ipynb
  - Add visualisation with pyldavis

## Topic Modelling on Programming Language Mixtures
Keypoint: topics are programming languages, file with mixture of programming languages, identify which is which.

Applicability to cyber-security: identifying malware embedded within normal programs (shellcode).

### Combining Source Files per Repository

TODO:
  - Copy work Sam's
  
  
### LDA Model

TODO:
  - Write model
  - Add visualisation with pyldavis
 

## ? Identifying more general topics/themes in programming languages ?

## Measuring the Efficacy of Topic Models

Main question: How do we evaluate how well a topic (from LDA for example) represents a meaningful "topic" or theme?

TODO: do some research on this??? There must be some papers etc that try to formalise this that we can borrow ideas from?

### Reading Tea Leaves Paper
http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Jaccard Index

### Comparing topic mixtures generated by model with measured mixtures

Idea:
  - save the % of each program langauge per repo. 
  - Then try to use LDA model to tell us "I believe repo <x> is 10% Topic 1, 20% Topic 2 etc". 
  - Use analysis from above two sections to create a "most likely mapping from lda topic to programming language".Then rate our model.

Here we can do cross-validation etc.